# EVPN-MPLS with Nornir Part 3: IRBs and L3VPN integrations
* This lab follows the evpn_mpls_with_nornir_part2.ipynb lab
* Lab is already configured with basic EVPN-MPLS from evpn_mpls_with_nornir_part2.ipynb

# EVPN-MPLS and IRBs
* Every PE can host an active IRB
* EVPN integrates with L3VPN
* All-Active design possibilities:
    * Automatic Gateway MAC-IP Synchronization
    * Manual Gateway MAC-IP Synchronization
    * Virtual Gateway Address

## Automatic Gateway MAC-IP Synchronization
* Simple config, but if a PE goes down, migrated endpoints can lose their gateway (**outage**)!
* Each IRB has a unique IP and unique MAC address
* Gateway IPs and MACs advertised via BGP
* IRB MAC/IP is advertised via Type 2 route with **'Default Gateway Community'**
    * PEs maintain a list of known remote gateway MAC addresses
    * PEs can intercept local traffic destined for a remote PE IRB MAC and process it locally preventing unneeded backhauling
* `show bridge evpn peer-gateway-macs`
* Manually configure MAC addresses in lab for easier readability
* Sample Config:
  
```
#PE-11:
set interfaces irb unit 150 family inet address 10.15.0.1/24
set interfaces irb unit 150 mac 00:00:10:15:00:01
set interfaces irb unit 250 family inet address 10.25.0.1/24
set interfaces irb unit 250 mac 00:00:10:25:00:01
set routing-instances VLAN_AWARE_EVI bridge-domains v150 routing-interface irb.150
set routing-instances VLAN_AWARE_EVI bridge-domains v250 routing-interface irb.250
set routing-instances VLAN_AWARE_EVI protocols evpn default-gateway advertise     <---default behavior, but good to make config explicit

#PE-22:
set interfaces irb unit 150 family inet address 10.15.0.2/24
set interfaces irb unit 150 mac 00:00:10:15:00:02
set interfaces irb unit 250 family inet address 10.25.0.2/24
set interfaces irb unit 250 mac 00:00:10:25:00:02
set routing-instances VLAN_AWARE_EVI bridge-domains v150 routing-interface irb.150
set routing-instances VLAN_AWARE_EVI bridge-domains v250 routing-interface irb.250
set routing-instances VLAN_AWARE_EVI protocols evpn default-gateway advertise

# Verify:
show bridge evpn peer-gateway-macs  <---VLAN Aware EVIs
show evpn  peer-mac-address         <---VLAN-Based EVIs
show evpn instance extensive
```

## Manual Gateway MAC-IP Synchronization
* Each IRB has a same IP and same MAC address
* Can make management and tshoot be harder
* Migrated endpoints stay up even when original PE goes down

```
#PE-11:
set interfaces irb unit 150 family inet address 10.15.0.1/24
set interfaces irb unit 150 mac 00:00:10:15:00:01
set interfaces irb unit 250 family inet address 10.25.0.1/24
set interfaces irb unit 250 mac 00:00:10:25:00:01
set routing-instances VLAN_AWARE_EVI bridge-domains v150 routing-interface irb.150
set routing-instances VLAN_AWARE_EVI bridge-domains v250 routing-interface irb.250
set routing-instances VLAN_AWARE_EVI protocols evpn default-gateway do-not-advertise   <---Don't send Type 2 route for this IRB

#PE-22:
set interfaces irb unit 150 family inet address 10.15.0.1/24
set interfaces irb unit 150 mac 00:00:10:15:00:01
set interfaces irb unit 250 family inet address 10.25.0.1/24
set interfaces irb unit 250 mac 00:00:10:25:00:01
set routing-instances VLAN_AWARE_EVI bridge-domains v150 routing-interface irb.150
set routing-instances VLAN_AWARE_EVI bridge-domains v250 routing-interface irb.250
set routing-instances VLAN_AWARE_EVI protocols evpn default-gateway do-not-advertise

# Verify:
show evpn instance extensive
```

## Virtual Gateway Address
* Most popular method
* Each IRB has two IP addresses:
    * Unique IP for management
        * MAC address normally left as default (not manually changed)
    * Shared IP for the default gateway (Virtual)
        * MAC address taken from VRRP MAC (but no VRRP configured)
        * Answers ARP reqeusts from CE

```
#PE-11:
set interfaces irb unit 150 family inet address 10.15.0.1/24 virtual-gateway-address 10.15.0.254
set interfaces irb unit 250 family inet address 10.25.0.1/24 virtual-gateway-address 10.25.0.254
set routing-instances VLAN_AWARE_EVI bridge-domains v150 routing-interface irb.150
set routing-instances VLAN_AWARE_EVI bridge-domains v250 routing-interface irb.250
set routing-instances VLAN_AWARE_EVI protocols evpn default-gateway no-gateway-community

#PE-22:
set interfaces irb unit 150 family inet address 10.15.0.2/24 virtual-gateway-address 10.15.0.254
set interfaces irb unit 250 family inet address 10.25.0.2/24 virtual-gateway-address 10.25.0.254
set routing-instances VLAN_AWARE_EVI bridge-domains v150 routing-interface irb.150
set routing-instances VLAN_AWARE_EVI bridge-domains v250 routing-interface irb.250
set routing-instances VLAN_AWARE_EVI protocols evpn default-gateway no-gateway-community     <---Advertise Type 2, but not Default Gateway Community

# Verify:
show arp <---Check from CE
```


In [5]:
import logging
import os
from nornir import InitNornir
from nornir_utils.plugins.functions import print_result
from nornir_napalm.plugins.tasks import napalm_get, napalm_cli, napalm_configure
from nornir.core.task import Task, Result

In [6]:
nr = InitNornir(config_file="nornir_inventory/config.yml")

In [7]:
def backup_all_configs(nr, backup_dir="backups", file_name_suffix=""):
    """Backs up running configurations from all devices in Nornir inventory."""
    os.makedirs(backup_dir, exist_ok=True)
    def backup_config(task):
        result = task.run(task=napalm_get, getters=["config"])
        config_data = result.result["config"]["running"]
        if not file_name_suffix:
            backup_filename = f"{backup_dir}/{task.host}_nornir_backup.conf"
        else:
            backup_filename = f"{backup_dir}/{task.host}_{file_name_suffix}_backup.conf"
        with open(backup_filename, "w") as backup_file:
            backup_file.write(config_data)
        print(f"Backup saved: {backup_filename}")
    nr.run(task=backup_config)

def restore_all_configs(nr, backup_dir="backups", file_name_suffix=""):
    """Restores all device configurations from the latest backup in the specified folder."""
    def restore_config(task):
        try:
            if not file_name_suffix:
                backup_filename = f"{backup_dir}/{task.host}_nornir_backup.conf"
            else:
                backup_filename = f"{backup_dir}/{task.host}_{file_name_suffix}_backup.conf"
            with open(backup_filename, "r") as file:
                config_data = file.read()
            task.run(task=napalm_configure, configuration=config_data, replace=True)
            print(f"Configuration restored for {task.host} from {backup_filename}")
        except Exception as err:
            print(f"No backup not found for {task.host}")
            print(err)
    nr.run(task=restore_config)

In [8]:
#backup_all_configs(nr, file_name_suffix="base_evpn_setup")
restore_all_configs(nr, file_name_suffix="base_evpn_setup")

Configuration restored for PE-22 from backups/PE-22_base_evpn_setup_backup.conf
Configuration restored for PE-11 from backups/PE-11_base_evpn_setup_backup.conf
Configuration restored for PE-33 from backups/PE-33_base_evpn_setup_backup.conf


In [29]:
# Helper functions for verification commands
def _run_verify_commands(task, commands):
    task.run(task=napalm_cli, commands=commands)
    
def verify_commands(commands):
    result = nr.run(_run_verify_commands, commands=commands)
    for device in result.keys():
        for command in result[device][1].result.keys():
            header = f"{device} - {command}:"
            delimiter = len(header) * '-'
            print(delimiter)
            print(header)
            print(delimiter)
            print(result[device][1].result[command])
    return result

## Configure Automatic Gateway MAC-IP Synchronization


In [14]:
nr = InitNornir(config_file="nornir_inventory/config.yml")
def evpn_irb_auto_gw_config(task):
    # Configure Automatic Gateway MAC-IP Synchronization
    irb_auto_gw_config = [
        f"set interfaces irb unit 150 family inet address 10.15.0.{task.host.data['ag_irb_final_octet']}/24",
        f"set interfaces irb unit 150 mac 00:00:10:15:00:0{task.host.data['ag_irb_final_octet']}",
        f"set interfaces irb unit 250 family inet address 10.25.0.{task.host.data['ag_irb_final_octet']}/24",
        f"set interfaces irb unit 250 mac 00:00:10:25:00:0{task.host.data['ag_irb_final_octet']}",
        "set routing-instances VLAN_AWARE_EVI bridge-domains v150 routing-interface irb.150",
        "set routing-instances VLAN_AWARE_EVI bridge-domains v250 routing-interface irb.250", "set routing-instances VLAN_AWARE_EVI protocols evpn default-gateway advertise ",
    ]
    try:
        irb_auto_gw_config = "\n".join(irb_auto_gw_config)
        task.run(task=napalm_configure, severity_level=logging.DEBUG, configuration=irb_auto_gw_config)
    except Exception as err:
        print("Error:", err)
        task.run(task=napalm_configure, severity_level=logging.DEBUG, configuration="rollback 0")

In [15]:
result = nr.run(evpn_irb_auto_gw_config)
print_result(result, severity_level=logging.DEBUG)

evpn_irb_auto_gw_config*********************************************************
* PE-11 ** changed : True ******************************************************
vvvv evpn_irb_auto_gw_config ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
---- napalm_configure ** changed : True ---------------------------------------- DEBUG
[edit interfaces]
+   irb {
+       unit 150 {
+           family inet {
+               address 10.15.0.1/24;
+           }
+           mac 00:00:10:15:00:01;
+       }
+       unit 250 {
+           family inet {
+               address 10.25.0.1/24;
+           }
+           mac 00:00:10:25:00:01;
+       }
+   }
[edit routing-instances VLAN_AWARE_EVI protocols evpn]
+     default-gateway advertise;
[edit routing-instances VLAN_AWARE_EVI bridge-domains v150]
+     routing-interface irb.150;
[edit routing-instances VLAN_AWARE_EVI bridge-domains v250]
  routing-interface irb.250;
^^^^ END evpn_irb_auto_gw_config ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [16]:
nr = InitNornir(config_file="nornir_inventory/config.yml")
def verify_irb_ag():
    irb_verification_commands = [
            "show bridge evpn peer-gateway-macs",
            "show evpn instance extensive VLAN_AWARE_EVI",
    ]
    return verify_commands(irb_verification_commands)

In [17]:
result = verify_irb_ag()

-------------------------------------------
PE-11 - show bridge evpn peer-gateway-macs:
-------------------------------------------


Routing instance : VLAN_AWARE_EVI
 Bridging domain : v150, VLAN : 150
  Installed GW MAC addresses:
  00:00:10:15:00:02  
  00:00:10:15:00:03  
 Bridging domain : v250, VLAN : 250
  Installed GW MAC addresses:
  00:00:10:25:00:02  
  00:00:10:25:00:03  

----------------------------------------------------
PE-11 - show evpn instance extensive VLAN_AWARE_EVI:
----------------------------------------------------

Instance: VLAN_AWARE_EVI
  Route Distinguisher: 10.11.11.11:1234
  Per-instance MAC route label: 299968
  Duplicate MAC detection threshold: 5
  Duplicate MAC detection window: 180
  MAC database status                     Local  Remote
    MAC advertisements:                       4       6
    MAC+IP advertisements:                    4       6
    Default gateway MAC advertisements:       2       4
  Number of local interfaces: 2 (2 up)
    Int

## Configure Manual Gateway MAC-IP Synchronization

In [21]:
#backup_all_configs(nr, file_name_suffix="base_evpn_setup")
restore_all_configs(nr, file_name_suffix="base_evpn_setup")

Configuration restored for PE-11 from backups/PE-11_base_evpn_setup_backup.conf
Configuration restored for PE-22 from backups/PE-22_base_evpn_setup_backup.conf
Configuration restored for PE-33 from backups/PE-33_base_evpn_setup_backup.conf


In [22]:
nr = InitNornir(config_file="nornir_inventory/config.yml")
def evpn_irb_manual_gw_config(task):
    # Configure manualmatic Gateway MAC-IP Synchronization
    irb_manual_gw_config = [
        "set interfaces irb unit 150 family inet address 10.15.0.1/24",
        "set interfaces irb unit 150 mac 00:00:10:15:00:01",
        "set interfaces irb unit 250 family inet address 10.25.0.1/24",
        "set interfaces irb unit 250 mac 00:00:10:25:00:01",
        "set routing-instances VLAN_AWARE_EVI bridge-domains v150 routing-interface irb.150",
        "set routing-instances VLAN_AWARE_EVI bridge-domains v250 routing-interface irb.250",
        "set routing-instances VLAN_AWARE_EVI protocols evpn default-gateway do-not-advertise",
    ]
    try:
        irb_manual_gw_config = "\n".join(irb_manual_gw_config)
        task.run(task=napalm_configure, severity_level=logging.DEBUG, configuration=irb_manual_gw_config)
    except Exception as err:
        print("Error:", err)
        task.run(task=napalm_configure, severity_level=logging.DEBUG, configuration="rollback 0")

In [23]:
result = nr.run(evpn_irb_manual_gw_config)
print_result(result, severity_level=logging.DEBUG)

evpn_irb_manual_gw_config*******************************************************
* PE-11 ** changed : True ******************************************************
vvvv evpn_irb_manual_gw_config ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
---- napalm_configure ** changed : True ---------------------------------------- DEBUG
[edit interfaces]
+   irb {
+       unit 150 {
+           family inet {
+               address 10.15.0.1/24;
+           }
+           mac 00:00:10:15:00:01;
+       }
+       unit 250 {
+           family inet {
+               address 10.25.0.1/24;
+           }
+           mac 00:00:10:25:00:01;
+       }
+   }
[edit routing-instances VLAN_AWARE_EVI protocols evpn]
+     default-gateway do-not-advertise;
[edit routing-instances VLAN_AWARE_EVI bridge-domains v150]
+     routing-interface irb.150;
[edit routing-instances VLAN_AWARE_EVI bridge-domains v250]
  routing-interface irb.250;
^^^^ END evpn_irb_manual_gw_config ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [24]:
nr = InitNornir(config_file="nornir_inventory/config.yml")
def verify_irb_manual():
    irb_verification_commands = [
            "show evpn instance extensive VLAN_AWARE_EVI",
    ]
    return verify_commands(irb_verification_commands)

In [25]:
result = verify_irb_manual()

----------------------------------------------------
PE-11 - show evpn instance extensive VLAN_AWARE_EVI:
----------------------------------------------------

Instance: VLAN_AWARE_EVI
  Route Distinguisher: 10.11.11.11:1234
  Per-instance MAC route label: 299968
  Duplicate MAC detection threshold: 5
  Duplicate MAC detection window: 180
  MAC database status                     Local  Remote
    MAC advertisements:                       4       4
    MAC+IP advertisements:                    4       2
    Default gateway MAC advertisements:       2       0
  Number of local interfaces: 2 (2 up)
    Interface name  ESI                            Mode             Status     AC-Role
    .local..12      00:00:00:00:00:00:00:00:00:00  single-homed     Up         Root 
    ge-0/0/2.1234   00:00:00:00:00:00:00:00:00:00  single-homed     Up         Root 
  Number of IRB interfaces: 2 (2 up)
    Interface name  VLAN   VNI    Status  L3 context
    irb.150         150            Up     master 

## Configure Virtual Gateway Address


In [26]:
#backup_all_configs(nr, file_name_suffix="base_evpn_setup")
restore_all_configs(nr, file_name_suffix="base_evpn_setup")

Configuration restored for PE-22 from backups/PE-22_base_evpn_setup_backup.conf
Configuration restored for PE-11 from backups/PE-11_base_evpn_setup_backup.conf
Configuration restored for PE-33 from backups/PE-33_base_evpn_setup_backup.conf


In [27]:
nr = InitNornir(config_file="nornir_inventory/config.yml")
def evpn_irb_virtual_gw_config(task):
    # Configure Virtual Gateway IRB
    irb_virtual_gw_config = [
        f"set interfaces irb unit 150 family inet address 10.15.0.{task.host.data['ag_irb_final_octet']}/24 virtual-gateway-address 10.15.0.254",
        f"set interfaces irb unit 150 mac 00:00:10:15:00:0{task.host.data['ag_irb_final_octet']}",
        f"set interfaces irb unit 250 family inet address 10.25.0.{task.host.data['ag_irb_final_octet']}/24 virtual-gateway-address 10.25.0.254",
        f"set interfaces irb unit 250 mac 00:00:10:25:00:0{task.host.data['ag_irb_final_octet']}",
        "set routing-instances VLAN_AWARE_EVI bridge-domains v150 routing-interface irb.150",
        "set routing-instances VLAN_AWARE_EVI bridge-domains v250 routing-interface irb.250", "set routing-instances VLAN_AWARE_EVI protocols evpn default-gateway advertise ",
    ]
    try:
        irb_virtual_gw_config = "\n".join(irb_virtual_gw_config)
        task.run(task=napalm_configure, severity_level=logging.DEBUG, configuration=irb_virtual_gw_config)
    except Exception as err:
        print("Error:", err)
        task.run(task=napalm_configure, severity_level=logging.DEBUG, configuration="rollback 0")

In [28]:
result = nr.run(evpn_irb_virtual_gw_config)
print_result(result, severity_level=logging.DEBUG)

evpn_irb_virtual_gw_config******************************************************
* PE-11 ** changed : True ******************************************************
vvvv evpn_irb_virtual_gw_config ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
---- napalm_configure ** changed : True ---------------------------------------- DEBUG
[edit interfaces]
+   irb {
+       unit 150 {
+           family inet {
+               address 10.15.0.1/24 {
+                   virtual-gateway-address 10.15.0.254;
+               }
+           }
+           mac 00:00:10:15:00:01;
+       }
+       unit 250 {
+           family inet {
+               address 10.25.0.1/24 {
+                   virtual-gateway-address 10.25.0.254;
+               }
+           }
+           mac 00:00:10:25:00:01;
+       }
+   }
[edit routing-instances VLAN_AWARE_EVI protocols evpn]
+     default-gateway advertise;
[edit routing-instances VLAN_AWARE_EVI bridge-domains v150]
+     routing-interface irb.150;
[edit routing-

#### Verify Virual Gateway IRB for EVPN-MPLS

* Manually configure a static route to due a quick test for the logical-system acting as the CE
* ARP show both gateways as expected :)
  
```
monty@PE-33:CE150> show route 
0.0.0.0/0          *[Static/5] 00:00:03
                    > to 10.15.0.254 via ge-0/0/3.150
10.15.0.0/24       *[Direct/0] 01:18:43
                    > via ge-0/0/3.150
10.15.0.33/32      *[Local/0] 01:18:43
                      Local via ge-0/0/3.150


monty@PE-33:CE150> show arp          
MAC Address       Address         Name                      Interface               Flags
00:00:10:15:00:03 10.15.0.3       10.15.0.3                 ge-0/0/3.150            none
50:00:00:07:00:05 10.15.0.11      10.15.0.11                ge-0/0/3.150            none
00:00:5e:00:01:01 10.15.0.254     10.15.0.254               ge-0/0/3.150            none
Total entries: 3
```